# 실습 6: Agent Runtime 배포

## 목표
- bedrock_agent_core.py 생성
- requirements.txt 생성
- Bedrock Agent Core Runtime 배포
- Runtime IAM Role에 권한 추가

## 6-1. 설정 확인

In [ ]:
import os
PROJECT_ROOT = os.getcwd()  # 프로젝트 루트 저장

from workshop_config import REGION, MEMORY_ID

print(f"🌍 Region: {REGION}")
print(f"🧠 Memory ID: {MEMORY_ID}")
print(f"📂 Project Root: {PROJECT_ROOT}")

if not MEMORY_ID:
    print("\n⚠️ MEMORY_ID가 없습니다. lab2를 먼저 실행하세요.")

---
## 6-2. bedrock_agent_core.py 생성

In [ ]:
%%writefile agent/deploy/runtime/bedrock_agent_core.py
"""Bedrock AgentCore Runtime Entrypoint"""

from bedrock_agentcore.runtime import BedrockAgentCoreApp
from agent.factory import create_agent

app = BedrockAgentCoreApp()


@app.entrypoint
async def integrated_disaster_agent(payload):
    """Handler for agent invocation"""
    userMessage = payload.get("prompt", "Hello! How can I assist you?")
    actorId = payload.get("actor_id", "default-user1")
    sessionId = payload.get("session_id", "default-session1")

    print(f"Processing request - actor: {actorId}, session: {sessionId}")

    agent = create_agent(actorId, sessionId)

    stream = agent.stream_async(userMessage)
    async for event in stream:
        yield event


if __name__ == "__main__":
    app.run()

---
## 6-3. requirements.txt 생성

In [ ]:
%%writefile agent/deploy/runtime/requirements.txt
strands-agents>=0.5.0
strands-agents-tools>=0.2.16
boto3>=1.34.0
bedrock-agentcore-starter-toolkit
bedrock-agentcore
wikipedia
requests
playwright>=1.40.0
nest-asyncio

---
## 6-4. Runtime 배포

In [ ]:
os.chdir(os.path.join(PROJECT_ROOT, 'agent/deploy/runtime'))
print(f"📂 현재 디렉토리: {os.getcwd()}")

In [ ]:
from config import REGION
from bedrock_agentcore_starter_toolkit import Runtime

print("🚀 Agent Runtime 배포 시작...\n")

agentcore_runtime = Runtime()

print("1️⃣ Runtime 설정 중...")
agentcore_runtime.configure(
    entrypoint="bedrock_agent_core.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=REGION,
    agent_name="agent_runtime"
)

print("\n2️⃣ Docker 빌드 및 배포 중... (5-10분)\n")
launch_result = agentcore_runtime.launch(auto_update_on_conflict=True)

print("\n✅ 배포 완료!")
print(f"📝 Agent ARN: {launch_result.agent_arn}")

AGENT_RUNTIME_ARN = launch_result.agent_arn

---
## 6-5. ⚠️ Runtime IAM Role 권한 추가 (필수)

Agent Runtime이 S3 Vectors, Bedrock 등 AWS 서비스에 접근하려면 IAM 권한이 필요합니다.

> **참고**: 실제로는 `s3vectors:QueryVectors` 등 최소 권한만 필요하지만, 워크샵 편의상 `AdministratorAccess`를 추가합니다.

In [ ]:
# Runtime Role 조회
import boto3

iam = boto3.client('iam')

# AmazonBedrockAgentCoreSDKRuntime-us-west-2 prefix로 시작하는 Role 찾기
ROLE_PREFIX = "AmazonBedrockAgentCoreSDKRuntime-us-west-2"

paginator = iam.get_paginator('list_roles')
matching_roles = []

for page in paginator.paginate():
    for role in page['Roles']:
        if role['RoleName'].startswith(ROLE_PREFIX):
            matching_roles.append(role['RoleName'])

print("🔍 발견된 Runtime Role:")
for i, role in enumerate(matching_roles, 1):
    print(f"   {i}. {role}")

if matching_roles:
    RUNTIME_ROLE_NAME = matching_roles[-1]  # 가장 최근 생성된 Role
    print(f"\n✅ 사용할 Role: {RUNTIME_ROLE_NAME}")
else:
    print("\n⚠️ Runtime Role을 찾을 수 없습니다.")

In [ ]:
# AdministratorAccess 정책 추가
try:
    iam.attach_role_policy(
        RoleName=RUNTIME_ROLE_NAME,
        PolicyArn='arn:aws:iam::aws:policy/AdministratorAccess'
    )
    print(f"✅ AdministratorAccess 정책이 {RUNTIME_ROLE_NAME}에 추가되었습니다.")
except Exception as e:
    print(f"⚠️ 권한 추가 실패: {e}")

---
## 6-6. workshop_config.py 업데이트

In [ ]:
os.chdir(PROJECT_ROOT)  # 프로젝트 루트로 복귀
print(f"📂 현재 디렉토리: {os.getcwd()}")

with open('workshop_config.py', 'r') as f:
    content = f.read()

content = content.replace('AGENT_RUNTIME_ARN = ""', f'AGENT_RUNTIME_ARN = "{AGENT_RUNTIME_ARN}"')

with open('workshop_config.py', 'w') as f:
    f.write(content)

print(f"✅ workshop_config.py 업데이트 완료")
print(f"📝 ARN: {AGENT_RUNTIME_ARN}")

---
## ✅ 실습 6 완료!

### 생성된 리소스
- Agent Runtime (Bedrock Agent Core)
- ECR Repository
- Docker Image
- Execution Role

### 생성된 파일
- `agent/deploy/runtime/bedrock_agent_core.py`
- `agent/deploy/runtime/requirements.txt`

### 업데이트된 파일
- `workshop_config.py` (AGENT_RUNTIME_ARN 추가)

### ⚠️ IAM 권한
- 워크샵 편의상 `AdministratorAccess` 추가
- 프로덕션에서는 `s3vectors:QueryVectors`, `bedrock:InvokeModel` 등 최소 권한 사용 권장

👉 다음: **lab7_web_application.ipynb**